In [1]:
#!pip install pyshp
import shapefile

In [ ]:
w = shapefile.Writer('shapefiles/test/polygon')
w.field('name', 'C')

w.poly([
[[113,24], [112,32], [117,36], [122,37], [118,20]], # poly 1
[[116,29],[116,26],[119,29],[119,32]], # hole 1
[[15,2], [17,6], [22,7]]  # poly 2
       ])
w.record('polygon1')
w.close()

In [ ]:
# Create a polygon shapefile
>>> w = shapefile.Writer(shapefile.POLYGON)
>>> w.poly(parts=[[[1,5],[5,5],[5,1],[3,3],[1,1]]])
>>> w.field('FIRST_FLD','C','40')
>>> w.field('SECOND_FLD','C','40')
>>> w.record('First','Polygon')
>>> w.save('shapefiles/test/polygon')